In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.vgg16 import VGG16
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout

import keras.backend as K
from typing import Optional

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [11]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 858 images belonging to 2 classes.


In [12]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 286 images belonging to 2 classes.


In [15]:
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


58889256/58889256 [==============================] - 0s 0us/step


In [16]:
for layer in base_model.layers:
    layer.trainable=False


In [17]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dense(1, activation='sigmoid'))

In [18]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [19]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) 

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [20]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [21]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [22]:
history=model.fit(train_dataset,
                        epochs = 20,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/20
27/27 [==============================] - 179s 6s/step - loss: 0.5108 - accuracy: 0.7529 - precision: 0.7868 - recall: 0.7672 - auc: 0.8385 - f1_score: 0.7764 - lr: 0.0100
Epoch 2/20
27/27 [==============================] - 15s 538ms/step - loss: 0.3826 - accuracy: 0.8333 - precision: 0.8611 - recall: 0.8378 - auc: 0.9055 - f1_score: 0.8481 - lr: 0.0063
Epoch 3/20
27/27 [==============================] - 15s 542ms/step - loss: 0.3508 - accuracy: 0.8683 - precision: 0.9017 - recall: 0.8586 - auc: 0.9193 - f1_score: 0.8795 - lr: 0.0040
Epoch 4/20
27/27 [==============================] - 15s 552ms/step - loss: 0.3102 - accuracy: 0.8648 - precision: 0.8942 - recall: 0.8607 - auc: 0.9388 - f1_score: 0.8745 - lr: 0.0025
Epoch 5/20
27/27 [==============================] - 15s 539ms/step - loss: 0.3408 - accuracy: 0.8625 - precision: 0.8837 - recall: 0.8690 - auc: 0.9263 - f1_score: 0.8732 - lr: 0.0016
Epoch 6/20
27/27 [==============================] - 15s 541ms/step - loss: 0.3071 

In [23]:
scores = model.evaluate(test_dataset)

9/9 [==============================] - 146s 18s/step - loss: 0.1936 - accuracy: 0.9196 - precision: 0.9717 - recall: 0.8374 - auc: 0.9744 - f1_score: 0.9005


In [24]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.9195803999900818
Precision =  0.9716981053352356
Recall =  0.8373983502388
AUC =  0.9743627905845642
F1_score =  0.9005255103111267
